# Lecture 4: Password CPA Attack - Introduction Solution

In [1]:
import securec
from securec import util
scope, target = util.init()

See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


In [2]:
securec.util.compile_and_flash('./4_password_fixed.c')

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2041 bytes
✓


In [3]:
import numpy as np

scope.default_setup()
scope.adc.samples = 500

def capture(attempt, count=1):
    if isinstance(attempt, str):
        attempt = attempt.encode('iso-8859-1')
    elif isinstance(attempt, int):
        attempt = bytes([attempt])
    traces = []
    for _ in range(count):
        scope.arm()
        target.simpleserial_write(0x01, attempt + b'\x00' * (10 - len(attempt)))
        result = target.simpleserial_read(0x01, 1)
        traces.append(util.capture())
    return np.mean(np.array(traces), axis=0), not bool(result[0])

In [4]:
import math
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool
from bokeh.palettes import Category10_10
from bokeh.models import Span, Label

output_notebook()

Loading BokehJS ...

## Exercise 2

In [5]:
traces = [(attempt, capture(attempt, count=10)[0]) for attempt in ('a', 'b', 'i', '\x00', '\xff')]

p = figure(height=500, sizing_mode='stretch_width', x_range=(15, 45))
p.add_tools(CrosshairTool())
for idx, (attempt, trace) in enumerate(traces):
    p.line(
        range(len(trace)),
        abs(trace),
        line_color=Category10_10[idx],
        legend_label=f'abs({attempt.encode("iso-8859-1")})',
    )

for i, (x, label) in enumerate(zip(
    range(21, 80, 4), 
    ('ld r24, Z+', '', 'ld r18, X+', '', 'eor r18, r24', 'or r25, r18')
)):
    if not label:
        continue
    p.add_layout(Span(location=x, dimension='height', line_color='darkslateblue', line_width=30, line_alpha=0.1))
    p.add_layout(Label(x=x, y=p.plot_height, text=label, y_units='screen', x_offset=-15, y_offset=-35,
                       text_align='right', text_color='darkslateblue', angle=math.pi/2))
    for idx, (_, trace) in enumerate(traces):
        p.circle(x, abs(trace)[x], size=10, color=Category10_10[idx])

show(p)

ERROR:ChipWhisperer Target:Device did not ack


## Exercise 3

In [6]:
import numpy as np

HW = [bin(n).count("1") for n in range(0, 256)]

def hw(n):
    if isinstance(n, str):
        return HW[ord(n)]
    return HW[n]

hw_vec = np.vectorize(hw)

In [7]:
traces = [(attempt, capture(attempt, count=10)[0]) for attempt in b'\x00\x01\x03\x07\x0f\x1f\x3f\x7f\xff']

p = figure(title='hw(attempt) vs trace @ ld r18, X+', sizing_mode='stretch_width', plot_height=400)
p.toolbar_location = None
for idx, (attempt, trace) in enumerate(traces):
    p.circle(
        hw(attempt),
        trace[33],
        color=Category10_10[idx],
        size=10,
        legend_label=f'{attempt:02x}',
    )

show(p)

In [8]:
traces = [(attempt, capture(attempt, count=1)[0]) for attempt in range(255)]

p = figure(title='hw(attempt ^ i) vs trace @ eor r18, r24', sizing_mode='stretch_width', plot_height=400)
p.toolbar_location = None
for idx, (attempt, trace) in enumerate(traces):
    p.circle(
        hw(attempt ^ ord('i')),
        trace[37],
        size=10,
    )

show(p)